# Setup

In [1]:
# Imporatamos librerias
import shutil
from PIL import Image, ImageDraw
from matplotlib.pyplot import imshow
import numpy as np
import glob
import os
import re
from datetime import datetime
from pathlib import Path

In [2]:
# Instalamos la librería de ultralytics que contiene YOLOv11
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 19.6 MB/s eta 0:00:00


In [3]:
# Creamos directorio de datasets y importamos ficheros de configuracion
!mkdir datasets
!cp drive/MyDrive/UOC/DELT/DELTv11.yaml .
!cp drive/MyDrive/UOC/DELT/.comet.config .

In [4]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
def full_name_func(source_folder,dataset,file_exclusion_patterns = [],subdirectories_included = [],frames = 4):

    full_name_list = []


    for root, dirs, files in os.walk(source_folder):
        for directory in dirs:
            if directory == 'images':
                source_images_path = os.path.join(root, directory)
                for file in os.listdir(source_images_path):
                    if not file.startswith(tuple(file_exclusion_patterns)):
                        src_file = os.path.join(source_images_path, file)
                        subfolder = os.path.basename(Path(src_file).parents[1])
                        if ((subfolder in subdirectories_included) or (not subdirectories_included)):
                            full_name = dataset + '-' + subfolder +'-'+ file
                            full_name_list.append(full_name[:-4])

    full_name_list.sort()

    final_list = [file for i,file in enumerate(full_name_list) if i%frames == 0]


    return final_list

In [6]:
def drive_to_colab(source_folder,destination_images,destination_labels,dataset,file_list):

    os.makedirs(destination_images, exist_ok=True)
    os.makedirs(destination_labels, exist_ok=True)


    for root, dirs, files in os.walk(source_folder):
        for directory in dirs:
            if directory == 'images':
                source_images_path = os.path.join(root, directory)
                for file in os.listdir(source_images_path):
                    src_file = os.path.join(source_images_path, file)
                    dst_file = os.path.join(destination_images, file)

                    subfolder = os.path.basename(Path(src_file).parents[1])
                    full_name = dataset + '-' + subfolder +'-'+ file
                    new_path = os.path.join(destination_images, full_name)

                    if full_name[:-4] in file_list:
                        shutil.copy(src_file, dst_file)
                        shutil.move(dst_file, new_path)
            elif directory == 'labels':
                source_labels_path = os.path.join(root, directory)
                for file in os.listdir(source_labels_path):
                    src_file = os.path.join(source_labels_path, file)
                    dst_file = os.path.join(destination_labels, file)

                    subfolder = os.path.basename(Path(src_file).parents[1])
                    full_name = dataset + '-' + subfolder +'-'+ file
                    new_path = os.path.join(destination_labels, full_name)

                    if full_name[:-4] in file_list:
                        shutil.copy(src_file, dst_file)
                        shutil.move(dst_file, new_path)

In [7]:
subs = []
for i in range(102,126):
  subs.append(str(i) + 'FTASK')
# Carpetas que no existen
for item in ['115FTASK','104FTASK','107FTASK','109FTASK','112FTASK',]:
  subs.remove(item)

# Carpetas usadas en train
for item in ['106FTASK','108FTASK','113FTASK','114FTASK','119FTASK','120FTASK','122FTASK','125FTASK']:
  subs.remove(item)

In [8]:
destination_images = '/content/datasets/DELT/val/images'
destination_labels = '/content/datasets/DELT/val/labels'

source_folder = '/content/drive/MyDrive/UOC/DELT/osaconservation/Arribada2023GT'

dataset = 'osaconservation'

n_frames = 1

s_included = subs
# s_included = []

source_folder = '/content/drive/MyDrive/UOC/DELT/' + dataset
file_list = full_name_func(source_folder,dataset,subdirectories_included = s_included,frames=n_frames)
drive_to_colab(source_folder,destination_images,destination_labels,dataset,file_list)


# OSA Conservation + YouTube (talla S)

In [ ]:
model_v = 'osaconservation-youtube-202412061719-yolo11s'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

model = YOLO(weigths_path)

pred_images =  destination_images + '/*.JPG'

project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

results = model(pred_images,project =project_folder, name =  model_v + '_predict',save = True,stream=False)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/2256 /content/datasets/DELT/val/images/osaconservation-102FTASK-IRX_0723.JPG: 512x640 (no detections), 437.2ms
image 2/2256 /content/datasets/DELT/val/images/osaconservation-102FTASK-IRX_0726.JPG: 512x640 (no detections), 414.3ms
image 3/2256 /content/datasets/DELT/val/images/osaconservation-102FTASK-IRX_0727.JPG: 512x640 (no detections), 440.4ms
image 4/2256 /content/datasets/DELT/val/images/osaconservation-102FTASK-IRX_0728.JPG: 512x640 (no d